## Pasos

Este cuaderno muestra cómo utilizar la nueva receta de personalización de usuarios de Amazon Personalize (aws-user-personalization). Esta receta equilibra las recomendaciones entre los artículos nuevos y los antiguos, permitiéndole ajustar el balance a favor de artículos más nuevos o más antiguos.

A grandes rasgos, la utilización de la nueva receta de personalización del USUARIO implica los siguientes pasos:  

1. Configurar clientes de Personalize
2. Crear un grupo de conjunto de datos, definir un esquema, importar conjuntos de datos e incorporar interacciones en tiempo real
4. Crear una campaña con una nueva configuración `campaignConfig`
5. Crear un rastreador de eventos para incorporar los eventos enviados por PutEvents
6. Llamar a GetRecommendations. Se devuelve un nuevo campo `RecommendationId` en la respuesta.
7. Llamar a putEvents con `RecommendationId` o una lista personalizada de artículos de impresión
8. Esperar a que se actualice la campaña
9. Actualizar la campaña para detener la actualización automática
10. Limpiar


> **NOTA:**: **La elaboración de este cuaderno llevará un par de horas.**

### Configuración de la clave de acceso/clave secreta para el acceso a la API de AWS.

Asegúrese de que la clave de acceso/clave secreta que utiliza tiene los permisos adecuados. Elija también la región en la que desea ejecutar esta demostración

In [ ]:
accessKeyId = ""
secretAccessKey = ""
region_name = ""

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError
import time
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [ ]:
suffix = str(np.random.uniform())[4:9]
prefix = 'user-personalization-'
print('prefix+suffix:{}{}'.format(prefix, suffix))
s3_bucket_name = (prefix + suffix).lower()
interaction_schema_name = prefix + 'interaction-'  + suffix
item_metadata_schema_name = prefix + 'items-'  + suffix
dataset_group_name = prefix + suffix
interaction_dataset_name = prefix + 'interactions-' + suffix
item_metadata_dataset_name = prefix + 'items-' + suffix
event_tracker_name = prefix + suffix
solution_name = prefix + suffix
event_tracker_name = prefix + suffix
campaign_name = prefix + suffix

### 1. Configuración del cliente
Primero, configuraremos el cliente para Personalize y S3.

In [ ]:
# Public s3 bucket owned by Personalize service which used to store the example dataset.

personalize_s3_bucket = "personalize-cli-json-models"
s3_client = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('s3')



####  Iniciar clientes de personalize

In [ ]:

personalize = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize')
personalize_runtime = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-runtime')
personalize_events = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-events')

### Conjuntos de datos de muestra

Por comodidad y para los fines de esta demostración, utilizaremos los conjuntos de datos de muestra proporcionados por Personalize.
Proporcionamos dos conjuntos de datos, uno de ellos son los metadatos de los artículos y el otro es un conjunto de datos de interacción. Primero los descargaremos de forma local.

In [ ]:
interaction_dataset_key = "sample-dataset/interactions-sample.csv"
items_dataset_key = "sample-dataset/items-with-creation-timestamp-sample.csv"
interactions_file = os.getcwd() + "/interaction_raw.csv"
items_metadata_file = os.getcwd() + "/items_raw.csv"
s3_client.download_file(personalize_s3_bucket, interaction_dataset_key, interactions_file)
s3_client.download_file(personalize_s3_bucket, items_dataset_key, items_metadata_file)

In [ ]:
interactions_df = pd.read_csv(interactions_file)
items_df = pd.read_csv(items_metadata_file)

#### Echemos un vistazo al marco de datos de la interacción.

In [ ]:
interactions_df.head(2)

#### Conjunto de datos de interacciones
**ITEM_ID**: Elemento correspondiente al EVENT_TYPE.

**EVENT_TYPE**: Tipo de evento.  

**TIMESTAMP (MARCA TEMPORAL)**: Marca temporal de la interacción en milisegundos. (Tenga en cuenta que está en milisegundos) 

**USER_ID**: Identificación de usuario correspondiente a esta impresión.  

**IMPRESIÓN:**  Ahora podría pasar opcionalmente los datos de la impresión junto con los datos del evento en el conjunto de datos de la interacción. Esto se pasa en un nuevo campo `IMPRESSION` como puede ver anteriormente, que toma una concatenación de los artículos con los que el usuario interactuó (por ejemplo, los artículos que se mostraron al usuario). Las impresiones también incluyen los artículos en los que se hizo clic.  


In [ ]:
items_df.head(2)

#### Conjunto de datos de artículos

El conjunto de datos de los artículos contiene los ID de los artículos y los metadatos asociados.

**ITEM_ID**: Identificación del artículo  

**géneros** Metadatos del elemento, si hay varias clasificaciones categóricas para el mismo elemento, utilice una canalización " | " para concatenar.

**creation_timestamp** Fecha en la que se agregó el elemento.

Puede encontrar más detalles sobre los conjuntos de datos en la documentación
https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

#### Actualizar la marca temporal

Aquí, vamos a actualizar la marca de nuestro conjunto de datos a 8 días a partir de hoy para mostrar el impacto de las interacciones en tiempo real en nuestras recomendaciones

In [ ]:
current_time = int(time.time())
one_hour_ago = current_time - 8 * 24 * 60 * 60
# Get the time gap between the latest timestamp in the interaction and the current time 
interactions_df = interactions_df.astype({"TIMESTAMP": 'int64'})
latest_time_in_csv = interactions_df["TIMESTAMP"].max()
delta = one_hour_ago - latest_time_in_csv

In [ ]:
# shift the latest timestamp in the interactions_df to be the last hour timestamp
interactions_df.TIMESTAMP = interactions_df.TIMESTAMP + delta
interactions_df.to_csv(os.getcwd() + "/interaction.csv", index = False)

# shift the latest timestamp in the items_df to be the last hour timestamp
items_df = items_df.astype({"creation_timestamp": 'int64'})
items_df.creation_timestamp = items_df.creation_timestamp + delta
items_df.to_csv(os.getcwd() + "/items.csv", index = False)

#### Después de la actualización, comprobaremos el conjunto de datos de interacción y de artículos una vez más

In [ ]:
interactions_df.head(2)

In [ ]:
items_df.head(2)


### 2. Incorporar datos a Amazon Personalize
Ahora, crearemos un grupo de conjunto de datos, un esquema y un trabajo de importación de conjuntos de datos y cargaremos los conjuntos de datos. Esto no ha cambiado.

#### a. Crear grupo de conjuntos de datos

Esto es similar a la receta existente, la documentación completa se puede encontrar [aquí](https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html)

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']

In [ ]:

print('dataset_group_arn : {}'.format(dataset_group_arn))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

#### b. Crear esquemas de conjuntos de datos

> **_NOTA:_**: `Impression` El campo tiene un tipo de cadena y utiliza una concatenación de canalizaciones para múltiples valores.

In [ ]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "IMPRESSION",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
    ],
    "version": "1.0"
}

In [ ]:
interaction_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)
# print(json.dumps(create_schema_response, indent=2))
interaction_schema_arn = interaction_schema_response['schemaArn']
print('interaction_schema_arn:\n', interaction_schema_arn)

In [ ]:
item_metadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRES",
        "type": "string",
        "categorical": True
    },
    {
        "name": "CREATION_TIMESTAMP",
        "type": "long"
    }
    ],
    "version": "1.0"
}

item_metadata_schema_response = personalize.create_schema(
    name = item_metadata_schema_name,
    schema = json.dumps(item_metadata_schema)
)

# print(json.dumps(create_schema_response, indent=2))
item_metadata_schema_arn = item_metadata_schema_response['schemaArn']
print('item_metadata_schema_arn:\n', item_metadata_schema_arn)

#### c. Crear conjuntos de datos

In [ ]:
interactions_dataset_response = personalize.create_dataset(
    datasetType = 'INTERACTIONS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn,
    name = interaction_dataset_name
)
interaction_dataset_arn = interactions_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('interaction_dataset_arn:\n', interaction_dataset_arn)

items_dataset_response = personalize.create_dataset(
    datasetType = 'ITEMS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_metadata_schema_arn,
    name = item_metadata_dataset_name
)
item_metadata_dataset_arn = items_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('item_metadata_dataset_arn:\n', item_metadata_dataset_arn)

#### d. Cargar los conjuntos de datos al bucket de S3, configurar la política del bucket de S3 adecuada, la función de IAM, etc.

Necesitamos cargar estos conjuntos de datos o puede proporcionar el nombre del bucket donde ya tiene los conjuntos de datos.

In [ ]:
#!aws s3 mb s3://{s3_bucket_name}
s3_bucket_name

In [ ]:
s3_client.create_bucket(Bucket=s3_bucket_name,
                          CreateBucketConfiguration={
                              'LocationConstraint': region_name})

In [ ]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [ ]:
s3_client.upload_file(Filename=interactions_file, Bucket=s3_bucket_name,
    Key="interaction.csv")
s3_client.upload_file(Filename=items_metadata_file, Bucket=s3_bucket_name,
    Key="items.csv")

#### e. Adjuntar la política a su bucket de S3

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(s3_bucket_name),
                "arn:aws:s3:::{}/*".format(s3_bucket_name)
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=s3_bucket_name, Policy=json.dumps(policy));

#### f. Configurar el rol IAM apropiado para que Personalize pueda acceder a los conjuntos de datos

In [ ]:
iam = boto3.client(service_name='iam', 
                         aws_access_key_id = accessKeyId, 
                         aws_secret_access_key = secretAccessKey)   


role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

In [ ]:
print('role_arn:', role_arn)

#### g. Crear trabajos de importación de conjuntos de datos para cargarlos

In [ ]:
time.sleep(20) # wait for RoleARN completion
interactions_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'interactions-dij-' + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'interaction.csv')
    },
    roleArn = role_arn
)

interactions_dij_arn = interactions_dij_response['datasetImportJobArn']
print('interactions_dij_arn: ', interactions_dij_arn)
#print(json.dumps(interactions_dij_arn, indent=2))

items_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'items-dij-' + suffix,
    datasetArn = item_metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'items.csv')
    },
    roleArn = role_arn
)

items_dij_arn = items_dij_response['datasetImportJobArn']
print('items_dij_arn:', items_dij_arn)

In [ ]:
dataset_job_arns = [interactions_dij_arn, items_dij_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and len(dataset_job_arns) != 0:
    time.sleep(60)    
    for dij_arn in dataset_job_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dij_arn
        )
        dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
        status = None
        if "latestDatasetImportJobRun" not in dataset_import_job:
            status = dataset_import_job["status"]
            print("{} : {}".format(dij_arn, status))
        else:
            status = dataset_import_job["latestDatasetImportJobRun"]["status"]
            print("DIJ_ARN: {}, LatestDatasetImportJobRun: {}".format(dij_arn, status))
    
        if status == "ACTIVE" or status == "CREATE FAILED":
            dataset_job_arns.remove(dij_arn)


### 3. Crear solución, SolutionVersion

Crearemos una solución con 'aws-user-personalization'. Esta receta equilibra las recomendaciones de artículos nuevos y antiguos que se entregan a los usuarios

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"
max_time = time.time() + 3*60*60 # 3 hours
create_solution_response = None
while time.time() < max_time:

    try:
        create_solution_response = personalize.create_solution(name=solution_name, 
                                    recipeArn= recipe_arn, 
                                    datasetGroupArn = dataset_group_arn)

        solution_arn = create_solution_response['solutionArn']
        print('solution_arn: ', solution_arn)
        break;
    except personalize.exceptions.ClientError as e:
        if 'EVENT_INTERACTIONS' not in str(e):
            print(json.dumps(create_solution_response, indent=2))
            print(e)
            break

#### Crear SolutionVersion

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 4. Crear una campaña
Cuando se crea la campaña, podemos establecer el valor itemExplorationConfig para configurar el peso de exploración de los artículos fríos y también el límite de edad de exploración. Por ahora, podemos establecer el peso de exploración en 0,9 y el límite de edad de exploración en 7, por lo que pensamos que todo el tiempo de creación de artículos en menos de 7 días se consideraría como artículo frío y haríamos una mayor exploración en esos nuevos artículos.

#### Crear una campaña

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = prefix + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.9",
            "explorationItemAgeCutOff": "7"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
describe_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)
campaign_summary = describe_campaign_response["campaign"]
campaign_summary

### 5. Llamar a GetRecommendations
A los efectos de la demostración, utilizaremos las identificaciones de usuario en el conjunto de datos de entrada para hacer las llamadas getRecommendation. 
> **_NOTA:_**: En la respuesta, tiene un nuevo campo `RecommendationId` que corresponde a la lista de artículos devueltos por Personalize GetRecommendations. Puede pasar este RecommendationId para indicar las impresiones.  
También podría pasar el valor Impression como una concatenación de cadenas de elementos. Si pasa tanto RecommendationId como ImpressionList, ImpressionList tendría prioridad y se utilizaría en el sistema.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')
print(rec_response['recommendationId'])

In [ ]:
rec_response['itemList']

### 6. Crear rastreador de eventos

Crea un rastreador de eventos que se utiliza cuando se envían datos de eventos al grupo de conjuntos de datos especificado mediante la API PutEvents.

In [ ]:
even_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = even_tracker_response['eventTrackerArn']
event_tracking_id = even_tracker_response['trackingId']
#print(json.dumps(even_tracker_response,indent=2))
print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))

### 7. Enviar los datos de impresión a Personalize a través de PutEvents
Amazon Personalize puede modelar dos tipos de impresiones:  
1. Impresiones implícitas e impresiones explícitas. Las impresiones implícitas son aquellas que se producen durante la sesión de un usuario y son registradas automáticamente por Amazon Personalize cada vez que al usuario se le muestra un artículo. Puede integrarlos en su flujo de trabajo de recomendación incluyendo el ID de recomendación (devuelto por las operaciones) como entrada para futuras peticiones PutEvents.  


2. Las impresiones explícitas son aquellas que se introducen manualmente cuando realiza una solicitud de PutEvents. Utilizaría impresiones explícitas cuando, por ejemplo, no muestre algunos de los elementos devueltos por GetRecommendations debido a la falta de disponibilidad, etc.   

> **NOTA:**: Si ha definido `impression` en su esquema de interacción como lo hizo anteriormente, necesita enviar la lista de impresiones (ya sea los artículos devueltos por GetRecommendations o los suyos).  
**Cuando el ID de las recomendaciones como las impresiones son , Amazon Personalize utilizará las impresiones explícitas por defecto.**



#### Pongamos el artículo recomendado anteriormente como impresiones

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
     'sentAt': datetime.now().timestamp(),
     'eventType' : 'click',
     'itemId' : rec_response['itemList'][0]['itemId'],        
     'recommendationId': rec_response['recommendationId'],
     'impression': [item['itemId'] for item in rec_response['itemList']],
     }]
    )

#### También podemos poner algunos artículos nuevos

Pongamos un nuevo itemId '2xx' a Personalize.

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '240',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '241',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '242',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '243',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '244',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '245',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '246',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '247',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '248',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '249',
     }]
    )

### 8. Crear una nueva SolutionVersion con updateMode

Después de realizar la solicitud PutEvents, espere alrededor de 15 minutos para que Personalize incorpore los nuevos datos. Luego, cree una nueva solutionVersion con el modo de actualización.

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn, trainingMode = "UPDATE")

solution_version_after_update = create_solution_version_response['solutionVersionArn']
print('solution_version_after_update:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_after_update
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 9. Actualizar la camapaña

Actualice la campaña con la última versión de la solución arn de actualización.

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, solutionVersionArn=solution_version_after_update)
print('campaign_arn_response: ', campaign_arn_response)

In [ ]:
# Wait for campaign update to reflect the new solution-version
solutionVersionArn = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    solutionVersionArn = describe_campaign_response["campaign"]["solutionVersionArn"]
    print("Campaign solution version: {}".format(solutionVersionArn))
    
    if solutionVersionArn == solution_version_after_update:
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']["solutionVersionArn"]
desc_campaign_response

### Después de actualizar la versión de la solución, hagamos la recomendación de nuevo

Esperaríamos que los nuevos artículos aparecieran en la recomendación, ya que fijamos la exploración alta como 0,9.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response['itemList']

### Actualizar la campaña con un peso de exploración diferente

Esperaríamos que aparecieran más artículos antiguos en la lista de recomendaciones, ya que fijamos un peso de exploración bajo.

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']
desc_campaign_response

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.1",
            "explorationItemAgeCutOff": "7"
        }
    })


In [ ]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.1":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

### Después de la exploración actualizada peso

Hagamos la recomendación de nuevo, deberíamos ver más artículos antiguos aquí.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response

#### 10. Eliminar recursos

Después de crear todos los recursos, limpiaremos todos los recursos.

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)
    
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)